In [1]:
import os
import glob
import random
import torch
import itertools
import datetime
import time
import sys
import argparse
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
from PIL import Image
from torchvision.utils import save_image, make_grid

sample_num = 10

# from_dataset = 'noisy_npy'
# to_dataset = 'bg_npy'
for from_dataset, to_dataset in zip(['noisy_npy','noisy_npy','bg_npy','bg_npy','clean_npy','clean_npy'],['bg_npy','clean_npy','noisy_npy','clean_npy','noisy_npy','bg_npy']):
    #########################################################################
    ############################  datasets.py  ###################################

    class ImageDataset(Dataset):
        def __init__(self, root, transforms_=None, unaligned=False, mode="train"):          
            self.transform = transforms.Compose(transforms_)                                ## transform to tensor
            self.unaligned = unaligned

            self.A = None
            self.B = None
            if mode == "train":
                self.each_label = 180
                for data_dir in root:
                    f_list = os.listdir(data_dir + '/'  + to_dataset)
                    for idx in range(min(sample_num, len(f_list))):
                        if idx%4==0:
                            continue
                        if self.A is None:
                            self.A = np.load(data_dir + '/'  + to_dataset + '/' + str(idx+1) + '.npy').astype('float')[:180]
                            self.B = np.load(data_dir + '/'  + from_dataset + '/' + str(idx+1) + '.npy').astype('float')[:180]
                        else:
                            self.A = np.concatenate((self.A, np.load(data_dir + '/'  + to_dataset + '/' + str(idx+1) + '.npy').astype('float')[:180]))
                            self.B = np.concatenate((self.B, np.load(data_dir + '/'  + from_dataset + '/' + str(idx+1) + '.npy').astype('float')[:180]))
            if mode == "test":
                self.each_label = 20
                for data_dir in root:
                    f_list = os.listdir(data_dir + '/'  + to_dataset)
                    for idx in range(min(sample_num, len(f_list))):
                        if self.A is None:
                            self.A = np.load(data_dir + '/'  + to_dataset + '/' + str(idx+1) + '.npy').astype('float')[180:200]
                            self.B = np.load(data_dir + '/'  + from_dataset + '/' + str(idx+1) + '.npy').astype('float')[180:200]
                        else:
                            self.A = np.concatenate((self.A, np.load(data_dir + '/'  + to_dataset + '/' + str(idx+1) + '.npy').astype('float')[180:200]))
                            self.B = np.concatenate((self.B, np.load(data_dir + '/'  + from_dataset + '/' + str(idx+1) + '.npy').astype('float')[180:200]))
            print(self.A.shape)
        def __getitem__(self, index):
            curve_A = self.A[index].astype(np.float32)               
            curve_B = self.B[index - (index % self.each_label) + random.randint(0, self.each_label-1)].astype(np.float32) 
            return {"A": curve_A, "B": curve_B}

        def __len__(self):
            return self.A.shape[0]



    #########################################################################
    ############################  models.py  ###################################

    def weights_init_normal(m):                                    
        classname = m.__class__.__name__                        
        if classname.find("Conv") != -1:                        
            torch.nn.init.normal_(m.weight.data, 0.0, 0.02)     
            if hasattr(m, "bias") and m.bias is not None:       
                torch.nn.init.constant_(m.bias.data, 0.0)      
        elif classname.find("BatchNorm1d") != -1:              
            torch.nn.init.normal_(m.weight.data, 1.0, 0.02)  
            torch.nn.init.constant_(m.bias.data, 0.0)       

    ##############################
    ##  ResidualBlock
    ##############################
    class ResidualBlock(nn.Module):
        def __init__(self, in_features):
            super(ResidualBlock, self).__init__()

            self.block = nn.Sequential(                     ## block = [pad + conv + norm + relu + pad + conv + norm]
                nn.ReflectionPad1d(1),                     
                nn.Conv1d(in_features, in_features, 3),    
                nn.InstanceNorm1d(in_features),            
                nn.ReLU(inplace=True),                      
                nn.ReflectionPad1d(1),                     
                nn.Conv1d(in_features, in_features, 3),  
                nn.InstanceNorm1d(in_features),          
            )

        def forward(self, x):                          
            return x + self.block(x)                   

    ##############################
    ##  GeneratorResNet
    ##############################
    class GeneratorResNet(nn.Module):
        def __init__(self, input_shape, num_residual_blocks):   ## (input_shape = (1, 1024), num_residual_blocks = 9)
            super(GeneratorResNet, self).__init__()

            channels = input_shape[0]                           ## channels = 3

            ## init
            out_features = 64                                   ## out_features = 64 
            model = [                                           ## model = [Pad + Conv + Norm + ReLU]
                nn.ReflectionPad1d(channels),                   ## ReflectionPad2d(3):
                nn.Conv1d(channels, out_features, 7, padding=3),           ## Conv2d(3, 64, 7)
                nn.InstanceNorm1d(out_features),                ## InstanceNorm2d(64):
                nn.ReLU(inplace=True),                          
            ]
            in_features = out_features                          ## in_features = 64

            ## Down sampling 2 times
            for _ in range(2):
                out_features *= 2                                                   ## out_features = 128 -> 256
                model += [                                                          ## (Conv + Norm + ReLU) * 2
                    nn.Conv1d(in_features, out_features, 3, stride=2, padding=1),
                    nn.InstanceNorm1d(out_features),
                    nn.ReLU(inplace=True),
                ]
                in_features = out_features                                          ## in_features = 256

            for _ in range(num_residual_blocks):
                model += [ResidualBlock(out_features)]                              ## model += [pad + conv + norm + relu + pad + conv + norm]

            # Up sampling 2 times
            for _ in range(2):
                out_features //= 2                                                  ## out_features = 128 -> 64
                model += [                                                          ## model += [Upsample + conv + norm + relu]
                    nn.Upsample(scale_factor=2),
                    nn.Conv1d(in_features, out_features, 3, stride=1, padding=1),
                    nn.InstanceNorm1d(out_features),
                    nn.ReLU(inplace=True),
                ]
                in_features = out_features                                          ## out_features = 64

            ## output                                                               ## model += [pad + conv + tanh]
            model += [nn.ReflectionPad1d(channels), nn.Conv1d(out_features, channels, 7), nn.Tanh()]    ## 

            self.model = nn.Sequential(*model)

        def forward(self, x):        
            x = x.unsqueeze(1)
            x = self.model(x)
            return x.squeeze(1)       

    ##############################
    #        Discriminator
    ##############################
    class Discriminator(nn.Module):
        def __init__(self, input_shape):                                        
            super(Discriminator, self).__init__()

            channels, height = input_shape                                     

            # Calculate output shape of image discriminator (PatchGAN)
            self.output_shape = (1, height // 2 ** 4)                  

            def discriminator_block(in_filters, out_filters, normalize=True):          
                """Returns downsampling layers of each discriminator block"""
                layers = [nn.Conv1d(in_filters, out_filters, 4, stride=2, padding=1)]   ## layer += [conv + norm + relu]    
                if normalize:                                                          
                    layers.append(nn.InstanceNorm1d(out_filters))
                layers.append(nn.LeakyReLU(0.2, inplace=True))
                return layers

            self.model = nn.Sequential(                                                 
                *discriminator_block(channels, 64, normalize=False),        ## layer += [conv(3, 64) + relu]
                *discriminator_block(64, 128),                              ## layer += [conv(64, 128) + norm + relu]
                *discriminator_block(128, 256),                             ## layer += [conv(128, 256) + norm + relu]
                *discriminator_block(256, 512),                             ## layer += [conv(256, 512) + norm + relu]
                # nn.ZeroPad1d((1, 0, 1, 0)),                                 ## layer += [pad]

            )
            self.model_conv = nn.Conv1d(512, 1, 4, padding=1)                             ## layer += [conv(512, 1)]

        def forward(self, img):            
            img = img.unsqueeze(1)
            img = self.model(img)
            img = F.pad(img, (1,0), "constant", 0)
            return self.model_conv(img).squeeze(1)       

    #########################################################################
    ############################  utils.py  ###################################

    class ReplayBuffer:
        def __init__(self, max_size=50):
            assert max_size > 0, "Empty buffer or trying to create a black hole. Be careful."
            self.max_size = max_size
            self.data = []

        def push_and_pop(self, data):                       
            to_return = []                                  
            for element in data.data:
                element = torch.unsqueeze(element, 0)
                if len(self.data) < self.max_size:          
                    self.data.append(element)
                    to_return.append(element)
                else:
                    if random.uniform(0, 1) > 0.5:          
                        i = random.randint(0, self.max_size - 1)
                        to_return.append(self.data[i].clone())
                        self.data[i] = element
                    else:
                        to_return.append(element)
            return Variable(torch.cat(to_return))


    class LambdaLR:                                
        def __init__(self, n_epochs, offset, decay_start_epoch):                                                ## (n_epochs = 50, offset = epoch, decay_start_epoch = 30)
            assert (n_epochs - decay_start_epoch) > 0, "Decay must start before the training session ends!"     ## n_epochs > decay_start_epoch 
            self.n_epochs = n_epochs
            self.offset = offset
            self.decay_start_epoch = decay_start_epoch

        def step(self, epoch):                                              ## return    1-max(0, epoch - 30) / (50 - 30)
            return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)





    ###########################################################################
    ############################  cycle_gan.py  ###################################

    parser = argparse.ArgumentParser()
    parser.add_argument("--epoch", type=int, default=0, help="epoch to start training from")
    parser.add_argument("--n_epochs", type=int, default=20, help="number of epochs of training")
    parser.add_argument("--dataset_name", type=str, default="curve", help="name of the dataset")## ../input/facades-dataset
    parser.add_argument("--batch_size", type=int, default=32, help="size of the batches")
    parser.add_argument("--lr", type=float, default=0.0003, help="adam: learning rate")
    parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
    parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
    parser.add_argument("--decay_epoch", type=int, default=3, help="epoch from which to start lr decay")
    parser.add_argument("--n_cpu", type=int, default=2, help="number of cpu threads to use during batch generation")
    parser.add_argument("--img_height", type=int, default=1024, help="size of image height")
    parser.add_argument("--img_width", type=int, default=256, help="size of image width")
    parser.add_argument("--channels", type=int, default=1, help="number of image channels")
    parser.add_argument("--sample_interval", type=int, default=100, help="interval between saving generator outputs")
    parser.add_argument("--checkpoint_interval", type=int, default=-1, help="interval between saving model checkpoints")
    parser.add_argument("--n_residual_blocks", type=int, default=9, help="number of residual blocks in generator")
    parser.add_argument("--lambda_cyc", type=float, default=10.0, help="cycle loss weight")
    parser.add_argument("--lambda_id", type=float, default=5.0, help="identity loss weight")
    # opt,unknow =parser.parse_know_args()
    opt = parser.parse_args(args=[])                 ## colab
    print(opt)

    os.makedirs("images/%s" % opt.dataset_name, exist_ok=True)
    os.makedirs("save/%s" % opt.dataset_name, exist_ok=True)

    ## input_shape:(1, 1024)
    input_shape = (opt.channels, opt.img_height)         

    G_AB = GeneratorResNet(input_shape, opt.n_residual_blocks)
    G_BA = GeneratorResNet(input_shape, opt.n_residual_blocks)
    D_A = Discriminator(input_shape)
    D_B = Discriminator(input_shape)

    criterion_GAN = torch.nn.MSELoss()
    criterion_cycle = torch.nn.L1Loss()
    criterion_identity = torch.nn.L1Loss()

    if torch.cuda.is_available():
        G_AB = G_AB.cuda()
        G_BA = G_BA.cuda()
        D_A = D_A.cuda()
        D_B = D_B.cuda()
        criterion_GAN.cuda()
        criterion_cycle.cuda()
        criterion_identity.cuda()
        
    if opt.epoch != 0:
        G_AB.load_state_dict(torch.load("save/%s/G_AB_%d.pth" % (opt.dataset_name, opt.epoch)))
        G_BA.load_state_dict(torch.load("save/%s/G_BA_%d.pth" % (opt.dataset_name, opt.epoch)))
        D_A.load_state_dict(torch.load("save/%s/D_A_%d.pth" % (opt.dataset_name, opt.epoch)))
        D_B.load_state_dict(torch.load("save/%s/D_B_%d.pth" % (opt.dataset_name, opt.epoch)))
    else:
        G_AB.apply(weights_init_normal)
        G_BA.apply(weights_init_normal)
        D_A.apply(weights_init_normal)
        D_B.apply(weights_init_normal)


    optimizer_G = torch.optim.Adam(
        itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=opt.lr, betas=(opt.b1, opt.b2)
    )
    optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
    optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

    ## 学习率更行进程
    lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(
        optimizer_G, lr_lambda=LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step
    )
    lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(
        optimizer_D_A, lr_lambda=LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step
    )
    lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(
        optimizer_D_B, lr_lambda=LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step
    )

    fake_A_buffer = ReplayBuffer()
    fake_B_buffer = ReplayBuffer()

    ## Training data loader 
    dataloader = DataLoader(       
        ImageDataset(["data"], unaligned=True), 
        batch_size=opt.batch_size,                                                              
        shuffle=True,
        num_workers=opt.n_cpu,
    )
    ## Test data loader
    val_dataloader = DataLoader(
        ImageDataset(["data"], unaligned=True, mode="test"), 
        batch_size=5,
        shuffle=True,
        num_workers=1,
    )


    import pandas as pd
    import matplotlib.pyplot as plt
    
    def sample_images(batches_done):      ## （100/200/300/400...）
        """保存测试集中生成的样本"""
        imgs = next(iter(val_dataloader))    
        G_AB.eval()
        G_BA.eval()
        real_A = Variable(imgs["A"]).cuda()    
        fake_B = G_AB(real_A)                  
        real_B = Variable(imgs["B"]).cuda()    
        fake_A = G_BA(real_B)                 

        fig ,ax=plt.subplots(5,4)
        for i in range(5):
            ax[i][0].plot(real_A.cpu()[i])
            ax[i][1].plot(fake_B.cpu().detach().numpy()[i])
            ax[i][2].plot(real_B.cpu()[i])
            ax[i][3].plot(fake_A.cpu().detach().numpy()[i])
        plt.savefig("images/%s/%s.png" % (opt.dataset_name, batches_done))


    def normalization(data):
        return (data - min(data)) / (max(data) - min(data))

    def train():
        # ----------
        #  Training
        # ----------
        prev_time = time.time()                             
        for epoch in range(opt.epoch, opt.n_epochs):        ## for epoch in (0, 50)
            for i, batch in enumerate(dataloader):          ## batch is a dict, batch['A']:(1, 3, 256, 256), batch['B']:(1, 3, 256, 256)
        #       print('here is %d' % i)
                real_A = Variable(batch["A"]).cuda()  ## A
                real_B = Variable(batch["B"]).cuda()  ## B

                valid = Variable(torch.ones((real_A.size(0), *D_A.output_shape)), requires_grad=False).cuda()     
                fake = Variable(torch.zeros((real_A.size(0), *D_A.output_shape)), requires_grad=False).cuda()    


                ## -----------------
                ##  Train Generator
                ## -----------------
                G_AB.train()
                G_BA.train()

                ## Identity loss                                              ## 
                loss_id_A = criterion_identity(G_BA(real_A), real_A)          ##
                loss_id_B = criterion_identity(G_AB(real_B), real_B)

                loss_identity = (loss_id_A + loss_id_B) / 2                   ## Identity loss 

                ## GAN loss
                fake_B = G_AB(real_A)                                         ## 
                loss_GAN_AB = criterion_GAN(D_B(fake_B), valid)               ## 
                fake_A = G_BA(real_B)                                         ## 
                loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)               ## 

                loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2                    ## GAN loss

                # Cycle loss                                                 
                recov_A = G_BA(fake_B)                                        ## 
                loss_cycle_A = criterion_cycle(recov_A, real_A)               ## 
                recov_B = G_AB(fake_A)
                loss_cycle_B = criterion_cycle(recov_B, real_B)

                loss_cycle = (loss_cycle_A + loss_cycle_B) / 2

                # Total loss                                                  ## 
                loss_G = loss_GAN + opt.lambda_cyc * loss_cycle + opt.lambda_id * loss_identity
                optimizer_G.zero_grad()                                       ##
                loss_G.backward()                                             ## 
                optimizer_G.step()                                            ## 


                ## -----------------------
                ## Train Discriminator A
                ## -----------------------
                loss_real = criterion_GAN(D_A(real_A), valid)
                
                fake_A_ = fake_A_buffer.push_and_pop(fake_A)
                loss_fake = criterion_GAN(D_A(fake_A_.detach()), fake)
                # Total loss
                loss_D_A = (loss_real + loss_fake) / 2
                optimizer_D_A.zero_grad()                                     ## 
                loss_D_A.backward()                                           ## 
                optimizer_D_A.step()                                          ## 

                ## -----------------------
                ## Train Discriminator B
                ## -----------------------
                
                loss_real = criterion_GAN(D_B(real_B), valid)         
                
                fake_B_ = fake_B_buffer.push_and_pop(fake_B)
                loss_fake = criterion_GAN(D_B(fake_B_.detach()), fake)
                # Total loss
                loss_D_B = (loss_real + loss_fake) / 2
                optimizer_D_B.zero_grad()                                     ## 
                loss_D_B.backward()                                           ## 
                optimizer_D_B.step()                                          ## 
                loss_D = (loss_D_A + loss_D_B) / 2


                ## ----------------------
                ##  Log Progress
                ## ----------------------

                batches_done = epoch * len(dataloader) + i                                        ## 
                batches_left = opt.n_epochs * len(dataloader) - batches_done                      ##
                time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))  ## 
                prev_time = time.time()
                # Print log
                sys.stdout.write(
                    "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f, adv: %f, cycle: %f, identity: %f] ETA: %s"
                    % (
                        epoch,
                        opt.n_epochs,
                        i,
                        len(dataloader),
                        loss_D.item(),
                        loss_G.item(),
                        loss_GAN.item(),
                        loss_cycle.item(),
                        loss_identity.item(),
                        time_left,
                    )
                )

            lr_scheduler_G.step()
            lr_scheduler_D_A.step()
            lr_scheduler_D_B.step()


        # torch.save(G_AB.state_dict(), "save/%s/G_AB_%d_%s_%s.pth" % (opt.dataset_name, epoch, to_dataset, from_dataset))
        torch.save(G_BA.state_dict(), "save/%s/G_BA_%d_%s_%s-30.pth" % (opt.dataset_name, 19, from_dataset, to_dataset))
        # torch.save(D_A.state_dict(), "save/%s/D_A_%d_%s.pth" % (opt.dataset_name, epoch, from_dataset))
        # torch.save(D_B.state_dict(), "save/%s/D_B_%d_%s.pth" % (opt.dataset_name, epoch, to_dataset))
        print("\nsave my model finished !!")
        
        #     if opt.checkpoint_interval != -1 and epoch % opt.checkpoint_interval == 0:
        #         # Save model checkpoints
        #         torch.save(G_AB.state_dict(), "saved_models/%s/G_AB_%d.pth" % (opt.dataset_name, epoch))
        #         torch.save(G_BA.state_dict(), "saved_models/%s/G_BA_%d.pth" % (opt.dataset_name, epoch))
        #         torch.save(D_A.state_dict(), "saved_models/%s/D_A_%d.pth" % (opt.dataset_name, epoch))
        #         torch.save(D_B.state_dict(), "saved_models/%s/D_B_%d.pth" % (opt.dataset_name, epoch))


    def test():
        
        parser = argparse.ArgumentParser()
        parser.add_argument('--batchSize', type=int, default=1, help='size of the batches')
        parser.add_argument('--dataroot', type=str, default='curve', help='root directory of the dataset')
        parser.add_argument('--channels', type=int, default=1, help='number of channels of input data')
        parser.add_argument('--n_residual_blocks', type=int, default=9, help='number of channels of output data')
        parser.add_argument('--size', type=int, default=1024, help='size of the data (squared assumed)')
        parser.add_argument('--cuda', type=bool, default=True, help='use GPU computation')
        parser.add_argument('--n_cpu', type=int, default=8, help='number of cpu threads to use during batch generation')
        parser.add_argument('--generator_A2B', type=str, default='save/curve/G_AB_19.pth', help='A2B generator checkpoint file')
        parser.add_argument('--generator_B2A', type=str, default='save/G_BA_19.pth', help='B2A generator checkpoint file')
        # opt,unknow =parser.parse_know_args()
        opt = parser.parse_args(args=[])                 ## colab
        print(opt)

        #################################
        ##          test               ##
        #################################

        ## input_shape:(1, 1024)
        input_shape = (opt.channels, opt.size) 
        
        netG_A2B = GeneratorResNet(input_shape, opt.n_residual_blocks)
        netG_B2A = GeneratorResNet(input_shape, opt.n_residual_blocks)

        if opt.cuda:
            netG_A2B.cuda()
            netG_B2A.cuda()

        netG_A2B.load_state_dict(torch.load(opt.generator_A2B))
        netG_B2A.load_state_dict(torch.load("save/%s/G_BA_%d_%s_%s-30.pth" % ('curve', 19, from_dataset, to_dataset)))

        netG_A2B.eval()
        netG_B2A.eval()

        Tensor = torch.cuda.FloatTensor if opt.cuda else torch.Tensor
        input_A = Tensor(opt.batchSize, opt.channels, opt.size)
        input_B = Tensor(opt.batchSize, opt.channels, opt.size)

        dataloader = DataLoader(ImageDataset(["data"], mode='test'), 
                                batch_size=opt.batchSize, shuffle=False, num_workers=opt.n_cpu)



        #################################
        ##           test              ##
        #################################

        if not os.path.exists('output/A'):
            os.makedirs('output/A')
        if not os.path.exists('output/B'):
            os.makedirs('output/B')

        generated_clean = []
        generated_noisy = []
        counter = 0

        from pathlib import Path
        path = Path(str.format("data/CycleGAN-{}-{}-30/", from_dataset, to_dataset))
        path.mkdir(parents=True, exist_ok=True)
        for i, batch in enumerate(dataloader):
            counter += 1
            ##  real
            real_A = Variable(input_A.copy_(batch['A'])).squeeze(0)
            real_B = Variable(input_B.copy_(batch['B'])).squeeze(0)
            ##  fake
            fake_B = 0.5*(netG_A2B(real_A).data + 1.0)
            fake_A = 0.5*(netG_B2A(real_B).data + 1.0)
            

            generated_clean.append(normalization(fake_A.cpu().numpy()[0]))
            generated_noisy.append(normalization(fake_B.cpu().numpy()[0]))

            if counter % 20 == 0:
                np.save(str.format("data/CycleGAN-{}-{}-30/{}", from_dataset, to_dataset, counter // 20), np.array(generated_clean))
                # np.save(str.format("data/CycleGAN-{}-{}/{}", to_dataset, from_dataset, counter // 200), np.array(generated_noisy[180:]))
                generated_clean = []
                generated_noisy = []

    train()  ## 
    test()   ## 
        # pid = os.getpid()
        # !kill -9 $pid

/home/visitor/anaconda3/envs/1DCNN/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Namespace(epoch=0, n_epochs=20, dataset_name='curve', batch_size=32, lr=0.0003, b1=0.5, b2=0.999, decay_epoch=3, n_cpu=2, img_height=1024, img_width=256, channels=1, sample_interval=100, checkpoint_interval=-1, n_residual_blocks=9, lambda_cyc=10.0, lambda_id=5.0)
(1260, 1024)
(200, 1024)


/home/visitor/anaconda3/envs/1DCNN/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32, 1, 64])) that is different to the input size (torch.Size([32, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[Epoch 0/20] [Batch 39/40] [D loss: 0.250855] [G loss: 0.977802, adv: 0.281871, cycle: 0.050791, identity: 0.037605] ETA: 0:01:22.379446

/home/visitor/anaconda3/envs/1DCNN/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([12, 1, 64])) that is different to the input size (torch.Size([12, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[Epoch 19/20] [Batch 39/40] [D loss: 0.235904] [G loss: 0.381299, adv: 0.271248, cycle: 0.007480, identity: 0.007051] ETA: 0:00:00.093257
save my model finished !!
Namespace(batchSize=1, dataroot='curve', channels=1, n_residual_blocks=9, size=1024, cuda=True, n_cpu=8, generator_A2B='save/curve/G_AB_19.pth', generator_B2A='save/G_BA_19.pth')
(200, 1024)
Namespace(epoch=0, n_epochs=20, dataset_name='curve', batch_size=32, lr=0.0003, b1=0.5, b2=0.999, decay_epoch=3, n_cpu=2, img_height=1024, img_width=256, channels=1, sample_interval=100, checkpoint_interval=-1, n_residual_blocks=9, lambda_cyc=10.0, lambda_id=5.0)
(1260, 1024)
(200, 1024)
[Epoch 19/20] [Batch 39/40] [D loss: 0.242549] [G loss: 0.356619, adv: 0.266363, cycle: 0.006086, identity: 0.005879] ETA: 0:00:00.094823
save my model finished !!
Namespace(batchSize=1, dataroot='curve', channels=1, n_residual_blocks=9, size=1024, cuda=True, n_cpu=8, generator_A2B='save/curve/G_AB_19.pth', generator_B2A='save/G_BA_19.pth')
(200, 1024)
N